In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Connected to GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected")

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model

In [ ]:
train_ds=image_dataset_from_directory(
    '/content/drive/MyDrive/sem 7/Lab/DL_Lab/Bean_Dataset',
    subset='training',
    validation_split=0.2,
    seed=123,
    image_size=(224,224),
    batch_size=32
)

val_ds=image_dataset_from_directory(
    '/content/drive/MyDrive/sem 7/Lab/DL_Lab/Bean_Dataset',
    subset='validation',
    validation_split=0.2,
    seed=123,
    image_size=(224,224),
    batch_size=32
)

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32)

    return image, {
        "class_output": tf.one_hot(label, depth=3),
        "bbox_output": bbox
    }

train_ds = train_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)


In [ ]:
base_model=VGG16(
    weights='imagenet',
    include_top=False,
    input_tensor=Input(shape=(224,224,3))
)

for layer in base_model.layers:
  layer.trainable=False

In [ ]:
x=Flatten()(base_model.output)

class_output=Dense(3,activation='softmax',name='class_output')(x)

bbox_output=Dense(4,activation='linear',name='bbox_output')(x)

In [ ]:
model=Model(inputs=base_model.input,outputs=[class_output,bbox_output])
model.compile(
    optimizer="adam",
    loss={"class_output": "categorical_crossentropy", "bbox_output": "mse"},
    metrics={"class_output": "accuracy", "bbox_output": "mse"}
)
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
import cv2

def show_prediction(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img) / 255.0
    img_input = np.expand_dims(img_array, axis=0)

    pred_class, pred_bbox = model.predict(img_input)

    # Get predicted class
    class_idx = np.argmax(pred_class[0])
    class_names = ["class1", "class2", "class3"]
    label = class_names[class_idx]
    score = np.max(pred_class[0])

    # Scale bbox back to image size
    xmin, ymin, xmax, ymax = pred_bbox[0]
    xmin, xmax = int(xmin*224), int(xmax*224)
    ymin, ymax = int(ymin*224), int(ymax*224)

    img_disp = np.array(img_array*255, dtype=np.uint8)
    img_disp = cv2.rectangle(img_disp, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
    cv2.putText(img_disp, f"{label} ({score:.2f})", (xmin, ymin-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

    plt.imshow(img_disp.astype("uint8"))
    plt.axis("off")
    plt.show()

show_prediction("/content/drive/MyDrive/sem 7/Lab/DL_Lab/Bean_Dataset/angular_leaf_spot/angular_leaf_spot_06.jpg")
